In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept
import scipy

from loading_data import get_data
from utils_maze import get_trials

In [ ]:
thisdir = os.getcwd()
dataloc = os.path.join(thisdir, 'cache', 'data')
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "speed")

In [ ]:
import info.r068d7 as r068d7
import info.r068d8 as r068d8
# infos = [r068d7, r068d8]
from run import spike_sorted_infos
infos = spike_sorted_infos

In [ ]:
def plot_run_thresh(info, thresh, t_smooth, filepath=None):
    events, position, _, _, _ = get_data(info)
    speed = position.speed(t_smooth=1.)

    s = nept.find_nearest_idx(position.time, info.task_times["pauseB"].stop-30)
    e = nept.find_nearest_idx(position.time, info.task_times["phase3"].stop)

    runs = nept.run_threshold(position, thresh=thresh, t_smooth=t_smooth)

    fig, ax = plt.subplots(figsize=(8,6))
    ax.plot(position[s:e].time, position.x[s:e], ms=3)
    ax.plot(position[s:e].time, position.y[s:e], ms=3)
    plt.plot(speed.time, speed.data, color="k")
    for start, stop in zip(runs.starts, runs.stops):
        ax.fill_between([start, stop], np.max([np.max(position.x), 
                                              np.max(position.y)]), 
                        color="k", alpha=0.2)
    plt.axhline(thresh, color="g")
    plt.xlim(info.task_times["phase3"].start+10, info.task_times["phase3"].start+150)
    plt.legend(["X", "Y", "Speed", "thresh"], bbox_to_anchor=(1.0, 1.0))
    
    plt.tight_layout()
   
    if filepath is not None:
        filename = info.session_id+"-run_thresh-"+str(thresh)+"-t_smooth-"+str(t_smooth)+".png"
        plt.savefig(os.path.join(filepath, filename))
        plt.close()
    else:
        plt.show()

In [ ]:
def plot_rest_thresh(info, thresh, t_smooth, filepath=None):
    print(info.session_id)
    print("Thresh:", thresh)
    print("t_smooth:", t_smooth)
    events, position, _, _, _ = get_data(info)
    speed = position.speed(t_smooth=1.)

    s = nept.find_nearest_idx(position.time, info.task_times["pauseB"].stop-30)
    e = nept.find_nearest_idx(position.time, info.task_times["phase3"].stop)

    runs = nept.rest_threshold(position, thresh=thresh, t_smooth=t_smooth)

    fig, ax = plt.subplots(figsize=(8,6))
    ax.plot(position[s:e].time, position.x[s:e], ms=3)
    ax.plot(position[s:e].time, position.y[s:e], ms=3)
    plt.plot(speed.time, speed.data, color="k")
    for start, stop in zip(runs.starts, runs.stops):
        ax.fill_between([start, stop], np.max([np.max(position.x), 
                                              np.max(position.y)]), 
                        color="k", alpha=0.2)
    plt.axhline(thresh, color="g")
    plt.xlim(info.task_times["phase3"].start+10, info.task_times["phase3"].start+150)
    plt.legend(["X", "Y", "Speed", "thresh"], bbox_to_anchor=(1.0, 0.9))
    
    plt.tight_layout()
    
    if filepath is not None:
        filename = info.session_id+"-rest_thresh-"+str(thresh)+"-t_smooth-"+str(t_smooth)+".png"
        plt.savefig(os.path.join(filepath, filename))
        plt.close()
    else:
        plt.show()

In [ ]:
for info in infos:
    plot_rest_thresh(info, thresh=12., t_smooth=0.8, filepath=output_filepath)

In [ ]:
for info in infos:
    plot_run_thresh(info, thresh=10., t_smooth=0.8, filepath=output_filepath)

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
events, position, _, _, _ = get_data(info)
xedges, yedges = nept.get_xyedges(position, binsize=8)
start = nept.find_nearest_idx(position.time, info.task_times["phase3"].start)
position = position[start:start+400]

In [ ]:
plt.plot(position.time, position.y, "k.")
plt.show()

In [ ]:
runs = nept.run_threshold(position, thresh=10., t_smooth=0.5)
running = position[runs]

In [ ]:
fig = plt.figure(figsize=(12, 10))

ax1 = plt.subplot()

xx, yy = np.meshgrid(xedges, yedges)

ax1.plot(position.x, position.y, "y.")

pad_amount = 5
ax1.set_xlim((np.floor(np.min(position.x))-pad_amount, np.ceil(np.max(position.x))+pad_amount))
ax1.set_ylim((np.floor(np.min(position.y))-pad_amount, np.ceil(np.max(position.y))+pad_amount))

n_timebins = running.n_samples

running_position, = ax1.plot([], [], "<", color="r")
rat_position, = ax1.plot([], [], "<", color="b")

fig.tight_layout()


def init():
    running_position.set_data([], [])
    rat_position.set_data([], [])
    return (running_position, rat_position)


def animate(i):
    running_position.set_data(running.x[i], running.y[i])
    rat_position.set_data(position.x[i], position.y[i])

    return (running_position, rat_position)

anim = animation.FuncAnimation(fig, animate, frames=n_timebins, interval=80, 
                               blit=False, repeat=False)

In [ ]:
HTML(anim.to_html5_video())